In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import requests
import json

import time
import re

from crpapi import CRP
from congress import Congress

## Support Functions

In [2]:
def get_keys(path):
    """
    Pulls necessary api keys from designated path
    """
    with open(path) as f:
        return json.load(f)

## ETL

In [54]:
current_congress = 116
start_congress = 113

### Voteview

In [4]:
def get_voteview_csv_content(content_type, current_congress_num):
    '''
    Retrieves congress data by content type (passed in) for the 104th Congress to the current Congress today.
    
    '''
    if content_type not in ['votes', 'rollcalls', 'members']:
        print('Not a valid content type')
        return
    
    for congress_num in range(104, current_congress_num+1):
        #download the vote each member made for each roll call
        url = 'https://voteview.com/static/data/out/{}/S{}_{}.csv'.format(content_type, congress_num, content_type)

        r = requests.get(url)
        content = r.content
        file = open('{}/S{}_{}.csv'.format(content_type, congress_num, content_type), 'wb')

        file.write(content)
        file.close()
        
        #put in a sleep timer to not overload the voteview servers
        time.sleep(1)
        
    return

In [18]:
# get_voteview_csv_content('votes', current_congress)

In [19]:
# get_voteview_csv_content('rollcalls', current_congress)

In [20]:
# get_voteview_csv_content('members', current_congress)

In [72]:
rollcall_df = pd.DataFrame()
for i in range(start_congress, current_congress+1):
    temp_df = pd.read_csv('rollcalls/S{}_rollcalls.csv'.format(i))
    rollcall_df = pd.concat([rollcall_df, temp_df], ignore_index=True)
    
rollcall_df.drop('dtl_desc', axis=1, inplace=True)
rollcall_df.dropna(axis=0, inplace=True)
    
display(rollcall_df.head())
display(rollcall_df.tail())
display(rollcall_df.info())

,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question
0,113,Senate,1,2013-01-24,1,1,78,16,0.522,-0.852,0.214,-0.031,-20.316,SRES15,Resolution Agreed to,A resolution to improve procedures for the con...,On the Resolution
1,113,Senate,2,2013-01-24,1,2,86,9,0.620,-0.785,0.201,-0.111,-17.612,SRES16,Resolution Agreed to,A resolution amending the Standing Rules of th...,On the Resolution
2,113,Senate,3,2013-01-28,1,3,35,62,0.277,-0.961,-0.305,0.050,-18.941,HR152,Amendment Rejected,To offset the cost of the bill with rescission...,On the Amendment
3,113,Senate,4,2013-01-28,1,4,63,36,0.195,-0.812,0.460,-0.392,-15.561,HR152,Bill Passed,A bill making supplemental appropriations for ...,On Passage of the Bill
4,113,Senate,5,2013-01-29,1,5,95,3,0.998,-0.068,1.749,-0.128,-9.990,PN42,Nomination Confirmed,"John Forbes Kerry, of Massachusetts, to be Sec...",On the Nomination


,congress,chamber,rollnumber,date,session,clerk_rollnumber,yea_count,nay_count,nominate_mid_1,nominate_mid_2,nominate_spread_1,nominate_spread_2,nominate_log_likelihood,bill_number,vote_result,vote_desc,vote_question
2261,116,Senate,504,2020-03-18,2,76,90,8,0.544,-0.839,0.268,-0.311,-13.629,HR6201,Bill Passed,A bill making emergency supplemental appropria...,On Passage of the Bill
2262,116,Senate,505,2020-03-22,2,77,47,47,-0.009,-0.717,-0.352,0.347,-5.644,HR748,Cloture on the Motion to Proceed Rejected,A bill to amend the Internal Revenue Code of 1...,On Cloture on the Motion to Proceed
2263,116,Senate,506,2020-03-23,2,78,49,46,-0.037,-0.497,-0.720,0.022,-2.314,HR748,Cloture on the Motion to Proceed Rejected,A bill to amend the Internal Revenue Code of 1...,On Cloture on the Motion to Proceed
2264,116,Senate,507,2020-03-25,2,79,48,48,-0.243,0.819,-0.516,-1.138,-6.480,HR748,Amendment Rejected,To ensure that additional unemployment benefit...,On the Amendment
2265,116,Senate,508,2020-03-25,2,80,96,0,0.000,0.000,0.000,0.000,0.000,HR748,Bill Passed,A bill to amend the Internal Revenue Code of 1...,On Passage of the Bill


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2263 entries, 0 to 2265
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   congress                 2263 non-null   int64  
 1   chamber                  2263 non-null   object 
 2   rollnumber               2263 non-null   int64  
 3   date                     2263 non-null   object 
 4   session                  2263 non-null   int64  
 5   clerk_rollnumber         2263 non-null   int64  
 6   yea_count                2263 non-null   int64  
 7   nay_count                2263 non-null   int64  
 8   nominate_mid_1           2263 non-null   float64
 9   nominate_mid_2           2263 non-null   float64
 10  nominate_spread_1        2263 non-null   float64
 11  nominate_spread_2        2263 non-null   float64
 12  nominate_log_likelihood  2263 non-null   float64
 13  bill_number              2263 non-null   object 
 14  vote_result             

None

In [91]:
votes_to_ignore = rollcall_df[rollcall_df['bill_number'].str.contains('PN')].bill_number.unique()

non_nomination_votes = rollcall_df[~rollcall_df['bill_number'].isin(votes_to_ignore)][['congress', 'bill_number']]

non_nomination_votes['bill_id'] = non_nomination_votes['bill_number'].apply(lambda x: x.lower()) + '-' + \
                                  non_nomination_votes['congress'].astype(str)

bills_to_search = non_nomination_votes['bill_id'].values
bills_to_search

array(['sres15-113', 'sres16-113', 'hr152-113', ..., 'hr748-116',
       'hr748-116', 'hr748-116'], dtype=object)

### Open Secrets

In [5]:
#get key for open secrets api
key_path = "/Users/flatironschool/.secret/open_secrets_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [6]:
'''To use if abstracted library fails me'''
# url = 'https://www.opensecrets.org/api/?method=candContrib'
# method = '?method=candContrib'
# params = {'cid' = 'N00003389',
#           'cycle' = 2014,
#           'output' = 'json'}

# r = requests.get(url+method, params=params, 

'To use if abstracted library fails me'

In [7]:
crp = CRP(api_key)

In [8]:
contribs = crp.candidates.contrib('N00003389', '2014')
contribs

[{'@attributes': {'org_name': 'Blackstone Group',
   'total': '184700',
   'pacs': '0',
   'indivs': '184700'}},
 {'@attributes': {'org_name': 'Goldman Sachs',
   'total': '129025',
   'pacs': '10000',
   'indivs': '119025'}},
 {'@attributes': {'org_name': 'Humana Inc',
   'total': '104500',
   'pacs': '10000',
   'indivs': '94500'}},
 {'@attributes': {'org_name': 'NorPAC',
   'total': '100151',
   'pacs': '-249',
   'indivs': '100400'}},
 {'@attributes': {'org_name': 'Kindred Healthcare',
   'total': '95450',
   'pacs': '10000',
   'indivs': '85450'}},
 {'@attributes': {'org_name': 'JPMorgan Chase & Co',
   'total': '93075',
   'pacs': '10000',
   'indivs': '83075'}},
 {'@attributes': {'org_name': 'Citigroup Inc',
   'total': '87100',
   'pacs': '10000',
   'indivs': '77100'}},
 {'@attributes': {'org_name': 'Alliance Coal',
   'total': '86600',
   'pacs': '5000',
   'indivs': '81600'}},
 {'@attributes': {'org_name': 'Votesane PAC',
   'total': '82000',
   'pacs': '0',
   'indivs': '82

In [11]:
industries = crp.candidates.industries('N00003389', '2014')
industries

[{'@attributes': {'industry_code': 'F07',
   'industry_name': 'Securities & Investment',
   'indivs': '2157107',
   'pacs': '270600',
   'total': '2427707'}},
 {'@attributes': {'industry_code': 'W06',
   'industry_name': 'Retired',
   'indivs': '1447510',
   'pacs': '0',
   'total': '1447510'}},
 {'@attributes': {'industry_code': 'E01',
   'industry_name': 'Oil & Gas',
   'indivs': '728710',
   'pacs': '380599',
   'total': '1109309'}},
 {'@attributes': {'industry_code': 'K01',
   'industry_name': 'Lawyers/Law Firms',
   'indivs': '761864',
   'pacs': '304650',
   'total': '1066514'}},
 {'@attributes': {'industry_code': 'H01',
   'industry_name': 'Health Professionals',
   'indivs': '734995',
   'pacs': '289500',
   'total': '1024495'}},
 {'@attributes': {'industry_code': 'F09',
   'industry_name': 'Insurance',
   'indivs': '647100',
   'pacs': '366500',
   'total': '1013600'}},
 {'@attributes': {'industry_code': 'F10',
   'industry_name': 'Real Estate',
   'indivs': '876975',
   'pacs

In [12]:
sector = crp.candidates.sector('N00003389', '2014')
sector

[{'@attributes': {'sector_name': 'Agribusiness',
   'sectorid': 'A',
   'indivs': '707620',
   'pacs': '531587',
   'total': '1239207'}},
 {'@attributes': {'sector_name': 'Communic/Electronics',
   'sectorid': 'B',
   'indivs': '683400',
   'pacs': '424243',
   'total': '1107643'}},
 {'@attributes': {'sector_name': 'Construction',
   'sectorid': 'C',
   'indivs': '476426',
   'pacs': '192450',
   'total': '668876'}},
 {'@attributes': {'sector_name': 'Defense',
   'sectorid': 'D',
   'indivs': '68950',
   'pacs': '204900',
   'total': '273850'}},
 {'@attributes': {'sector_name': 'Energy/Nat Resource',
   'sectorid': 'E',
   'indivs': '1282860',
   'pacs': '760082',
   'total': '2042942'}},
 {'@attributes': {'sector_name': 'Finance/Insur/RealEst',
   'sectorid': 'F',
   'indivs': '5123131',
   'pacs': '1198450',
   'total': '6321581'}},
 {'@attributes': {'sector_name': 'Health',
   'sectorid': 'H',
   'indivs': '1691266',
   'pacs': '1057523',
   'total': '2748789'}},
 {'@attributes': {'

In [10]:
help(crp.candidates)

Help on CandidatesClient in module crpapi object:

class CandidatesClient(Client)
 |  CandidatesClient(apikey=None, cache='.cache')
 |  
 |  Retrieves and parses information pertaining to current Congressional
 |  legislators.
 |  
 |  Method resolution order:
 |      CandidatesClient
 |      Client
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  contrib(self, cid, cycle=None)
 |  
 |  contrib_by_ind(self, cid, industry, cycle=None)
 |  
 |  get(self, id_code)
 |      id_code may be either a candidate's specific CID, or a two letter
 |      state code, or a four character district code.
 |  
 |  industries(self, cid, cycle=None)
 |  
 |  pfd(self, cid, year=None)
 |  
 |  sector(self, cid, cycle=None)
 |  
 |  summary(self, cid, cycle=None)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Client:
 |  
 |  __init__(self, apikey=None, cache='.cache')
 |      Initialize self.  See help(type(self)) for accurate signa

### MIT Election Lab

Having trouble finding the right structure for this API call for a download.

In [21]:
#get key for dataverse api
key_path = "/Users/flatironschool/.secret/dataverse_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [31]:
server_url = 'https://dataverse.harvard.edu/dataset'
path = '/api/access/datafile/'
data_id = 'data'
# data_id = ':persistentId/?persistentId=doi:10.7910/DVN/PEJ5QU'

# '$SERVER_URL/api/search?q=$QUERY'

# 'GET http://$SERVER/api/access/datafile/:persistentId/?persistentId=doi:10.5072/FK2/J8SJZB'

url = server_url + path + data_id

r = requests.get(url)
print(r)
print(r.url)
# print(r.content)
print(r.text[:1000])

<Response [404]>
https://dataverse.harvard.edu/dataset/api/access/datafile/data
<?xml version='1.0' encoding='UTF-8' ?>
<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en"><head id="j_idt2"><!-- Global site tag (gtag.js) - Google Analytics -->
<script async="async" src="https://www.googletagmanager.com/gtag/js?id=UA-61753334-1"></script>
<script>
  //<![CDATA[
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date()); gtag('config', 'UA-61753334-1');

  window.addEventListener("load", enableAnalyticsEventCapture, false);

  function enableAnalyticsEventCapture() {
    // Download button
    $(document).on("click", ".btn-download", function() {
      var category = $(this).text();
      var label = getFileId($(this));
      gtag('event', 'Download',{'event_category' : category,
                                'event_label' : label});
    });

    // Request Access button
    $(document).on("click", 

### ProPublica

In [29]:
key_path = "/Users/flatironschool/.secret/pro_publica_api.json"
keys = get_keys(key_path)

api_key = keys['api_key']

In [30]:
congress = Congress(api_key)

In [34]:
member_test = congress.members.get('M000355')
member_test

{'id': 'M000355',
 'member_id': 'M000355',
 'first_name': 'Mitch',
 'middle_name': None,
 'last_name': 'McConnell',
 'suffix': None,
 'date_of_birth': '1942-02-20',
 'gender': 'M',
 'url': 'https://www.mcconnell.senate.gov',
 'times_topics_url': 'http://topics.nytimes.com/top/reference/timestopics/people/m/mitch_mcconnell/index.html',
 'times_tag': 'McConnell, Mitch (Per)',
 'govtrack_id': '300072',
 'cspan_id': '2351',
 'votesmart_id': '53298',
 'icpsr_id': '14921',
 'twitter_account': 'McConnellPress',
 'facebook_account': 'mitchmcconnell',
 'youtube_account': None,
 'crp_id': 'N00003389',
 'google_entity_id': '/m/01z6ls',
 'rss_url': 'https://www.mcconnell.senate.gov/public/?a=RSS.Feed',
 'in_office': True,
 'current_party': 'R',
 'most_recent_vote': '2020-03-26',
 'last_updated': '2020-04-22 09:45:55 -0400',
 'roles': [{'congress': '116',
   'chamber': 'Senate',
   'title': 'Senator, 2nd Class',
   'short_title': 'Sen.',
   'state': 'KY',
   'party': 'R',
   'leadership_role': 'Sen

In [36]:
bill_summary = congress.bills.get('SJRES2')
bill_summary

{'bill_id': 'sjres2-116',
 'bill_slug': 'sjres2',
 'congress': '116',
 'bill': 'S.J.RES.2',
 'bill_type': 'sjres',
 'number': 'S.J.RES.2',
 'bill_uri': 'https://api.propublica.org/congress/v1/116/bills/sjres2.json',
 'title': "A joint resolution disapproving the President's proposal to take an action relating to the application of certain sanctions with respect to the Russian Federation.",
 'short_title': "A joint resolution disapproving the President's proposal to take an action relating to the application of certain sanctions with respect to the Russian Federation.",
 'sponsor_title': 'Sen.',
 'sponsor': 'Charles E. Schumer',
 'sponsor_id': 'S000148',
 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/S000148.json',
 'sponsor_party': 'D',
 'sponsor_state': 'NY',
 'gpo_pdf_uri': None,
 'congressdotgov_url': 'https://www.congress.gov/bill/116th-congress/senate-joint-resolution/2',
 'govtrack_url': 'https://www.govtrack.us/congress/bills/116/sjres2',
 'introduced_date': '20

In [37]:
bill_subject = congress.bills.subjects('SJRES2')
bill_subject

{'congress': '116',
 'bill_id': 'sjres2-116',
 'bill_slug': 'sjres2',
 'bill_type': 'sjres',
 'number': 'S.J.RES.2',
 'bill_uri': 'https://api.propublica.org/congress/v1/116/bills/sjres2.json',
 'url_number': 'sjres2',
 'title': "A joint resolution disapproving the President's proposal to take an action relating to the application of certain sanctions with respect to the Russian Federation.",
 'sponsor_title': 'Sen.',
 'sponsor_id': 'S000148',
 'sponsor_name': 'Charles E. Schumer',
 'sponsor_state': 'NY',
 'sponsor_party': 'D',
 'sponsor_uri': 'https://api.propublica.org/congress/v1/members/S000148.json',
 'introduced_date': '2019-01-04',
 'number_of_cosponsors': 0,
 'committees': 'Senate Banking, Housing, and Urban Affairs Committee',
 'latest_major_action_date': '2019-01-16',
 'latest_major_action': 'Returned to the Calendar. Calendar No. 13.',
 'house_passage_vote': None,
 'senate_passage_vote': None,
 'subjects': [{'name': 'Subversive activities',
   'url_name': 'subversive-activit

In [38]:
bill_text = congress.bills.get('SJRES2')

In [35]:
help(congress.bills)

Help on BillsClient in module congress.bills object:

class BillsClient(congress.client.Client)
 |  BillsClient(apikey=None, cache='.cache', http=None)
 |  
 |  Client classes deal with fetching responses from the ProPublica Congress API
 |  and parsing what comes back. In addition to storing API credentials, a client
 |  can use a custom cache, or even a customized httplib2.Http instance.
 |  
 |  Method resolution order:
 |      BillsClient
 |      congress.client.Client
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  amendments(self, bill_id, congress=116)
 |  
 |  by_member(self, member_id, type='introduced')
 |      Takes a bioguide ID and a type (introduced|updated|cosponsored|withdrawn), 
 |      returns recent bills
 |  
 |  cosponsors(self, bill_id, congress=116)
 |  
 |  get(self, bill_id, congress=116, type=None)
 |  
 |  introduced(self, chamber, congress=116)
 |      Shortcut for getting introduced bills
 |  
 |  major(self, chamber, congress=116)
 |      S